In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import mysql.connector
schoolTable = pd.DataFrame()
# div class='sXLaOe'

In [8]:
df = pd.read_excel('./文件集/學校統整V2.xlsx')
len(df['校名'].unique())

123

### 學校網址_地區_縣市

In [11]:
dfSchool = pd.read_excel("./文件集/大專院校代碼v2.xlsx")
dfSummary = pd.read_excel("./文件集/學校統整V3.xlsx")

In [4]:
cols = dfSchool.iloc[1].to_list()
dfSchool.columns = cols
dfSchool = dfSchool[2:len(dfSchool)].reset_index().drop(['index'], axis=1)
dfSchool.set_index('學校名稱', inplace=True)

In [5]:
dicSiteAndLocs = {}
for key in dfSummary['校名'].unique():
    try:
        dicSiteAndLocs[key] = dfSchool.loc[key][['縣市別', '第三級行政區', '郵遞區號', '網址']].tolist()
    except:
        dicSiteAndLocs[key] = [0] * 4

In [113]:
# key = '中華大學'
# index = '中華大學學校財團法人中華大學'
# dicSiteAndLocs[key] = dfSchool.loc[index][['縣市別', '第三級行政區', '郵遞區號', '網址']].tolist()

In [16]:
newData = [[], [], [], []]
newCols = ['縣市', '行政區', '郵政區號', '網址']
for school in dfSummary['校名']:
    data = dicSiteAndLocs[school]
    for i in range(4):
        newData[i].append(data[i])
for i in range(4):
     dfSummary[newCols[i]] = newData[i]

In [17]:
dfSummary.to_excel("./文件集/學校統整V3.xlsx", index=False)

### 爬蟲尋找學校地址

In [3]:
def findSchoolAndCampus(schooldf):
    schoolNames = []
    schoolNames.append(schooldf.iloc[0]['校名'])
    for i in range(1, len(schooldf)):
        schoolName = schooldf.iloc[i]['校名']
        campus = schooldf.iloc[i]['校區']
        if(campus != '0'):
            schoolName += campus
        if(len(schoolNames) > 0 and (schoolNames[len(schoolNames)-1] != schoolName)):
            schoolNames.append(schoolName)
    return schoolNames

In [4]:
df = pd.read_excel('./文件集/學校統整.xlsx')
schoolNames = findSchoolAndCampus(df)

In [6]:
addr = []
for schoolName in schoolNames:
    response = requests.get("https://www.google.com/search?q=" + schoolName + "地址")
    soup = BeautifulSoup(response.text, "html.parser")
    # print(soup.prettify())
    select = soup.find_all("div","BNeawe iBp4i AP7Wnd")
    if(len(select) > 0):
        addr.append(select[0].text)
    else:
        addr.append("0")
dicAddr = {schoolNames[i] : addr[i] for i in range(len(schoolNames))}

In [14]:
addresses = []
for i in range(len(df)):
    schoolName = df.iloc[i]['校名']
    campus = df.iloc[i]['校區']
    if(campus != "0"):
        schoolName += campus
    addresses.append(dicAddr[schoolName])

In [17]:
df['地址'] = addresses
df.to_excel("./文件集/學校統整.xlsx", index=False)

### 彙整校區

In [6]:
def findCampus(pdSeries):
    campusArr = []
    for i in range(len(pdSeries)):
        campus = ''
        for j in range(len(pdSeries.iloc[i])):
            if(pdSeries.iloc[i][j] == "(" or pdSeries.iloc[i][j] == "（"):
                for k in range(j+1, len(pdSeries.iloc[i])):
                    if(pdSeries.iloc[i][k] == ")" or pdSeries.iloc[i][k] == "）"):
                        break
                    campus += pdSeries.iloc[i][k]
                break
        campusArr.append(campus)
    return pd.DataFrame(data=campusArr, index=pdSeries.index, columns=['校區'])

In [7]:
df = pd.read_excel("./文件集/學校統整.xlsx")

In [18]:
mask = df['科系名稱'].str.find("校區") > -1
campusDataframe = findCampus(df[mask]['科系名稱'])
schooldf = pd.concat([df, campusDataframe], axis=1)
schooldf.fillna('0', inplace=True)

In [ ]:
schooldf.to_excel("./文件集/學校統整.xlsx", index=False)

### 彙整學校資訊

In [36]:
df = pd.read_excel("./文件集/112技優保送.xls")
dfTemp = df[['學校代碼' ,'學校名稱', '系科組學程名稱']]
dfTemp.columns = ['學校ID', '校名', '科系名稱']
dfTemp

,學校ID,校名,科系名稱
0,101,國立臺灣科技大學,機電技優專班
1,101,國立臺灣科技大學,機電技優專班
2,101,國立臺灣科技大學,機電技優專班
3,102,國立雲林科技大學,跨域整合設計學士學位學程（設計技優專班）
4,102,國立雲林科技大學,智慧機器人學士學位學程（機器人技優專班）
...,...,...,...
147,712,國立高雄師範大學,工業科技教育學系能源與冷凍空調組
148,719,國立暨南國際大學,國際企業學系
149,802,輔仁大學,織品服裝學系服飾設計組
150,804,中原大學,電機工程學系


In [37]:
schoolTable = pd.concat([schoolTable, dfTemp], ignore_index=True)
schoolTable

,學校ID,校名,科系名稱
0,101,國立臺灣科技大學,機械工程系
1,101,國立臺灣科技大學,機械工程系
2,101,國立臺灣科技大學,材料科學與工程系
3,101,國立臺灣科技大學,材料科學與工程系
4,101,國立臺灣科技大學,材料科學與工程系
...,...,...,...
4842,712,國立高雄師範大學,工業科技教育學系能源與冷凍空調組
4843,719,國立暨南國際大學,國際企業學系
4844,802,輔仁大學,織品服裝學系服飾設計組
4845,804,中原大學,電機工程學系


In [40]:
schoolTable.to_excel("./文件集/學校統整.xlsx", index=False)

In [ ]:
mask = df['系名'].str.find("校區") > -1
df[mask][['校名', '系名']]

### mysql連線

In [ ]:
conn = mysql.connector.connect(user='project_tester', password='password', host='localhost', database='final_project')
cursor = conn.cursor()
cursor.execute("show tables;")
for row in cursor:
    print(row)